In [25]:
import pandas as pd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
from segment_speed_utils import helpers, time_series_utils
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

i'd start with 5, checking if the columns you want are already there in this file (saved out at the end of stop_arrivals_to_speed, and is a speeds by trip table for that day)

In [28]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [29]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id', 'route_id', 'route_short_name'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '${speed_vars.time_min_cutoff}', 'max_trip_minutes': 180, 'max_speed': '${speed_vars.max_speed}', 'min_meters_elaps

In [30]:
analysis_date

'2024-08-14'

In [31]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [32]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/speedmap/speeds_2024-08-14.parquet'

In [33]:
df = pd.read_parquet(path)

In [34]:
df.columns

Index(['trip_instance_key', 'stop_sequence', 'stop_sequence1',
       'shape_array_key', 'stop_meters', 'arrival_time', 'arrival_time_sec',
       'subseq_arrival_time_sec', 'subseq_stop_meters', 'meters_elapsed',
       'sec_elapsed', 'speed_mph', 'schedule_gtfs_dataset_key', 'route_id',
       'direction_id', 'route_short_name', 'scheduled_service_minutes',
       'sched_rt_category', 'time_of_day', 'shape_id', 'segment_id',
       'stop_pair_name', 'stop_pair'],
      dtype='object')

In [36]:
route_info = helpers.import_scheduled_trips(
    analysis_date,
    columns=[
        "gtfs_dataset_key", "trip_instance_key",
        "route_id", "direction_id", "route_short_name"
    ],
    get_pandas=True,
)

In [37]:
route_info

,schedule_gtfs_dataset_key,trip_instance_key,route_id,direction_id,route_short_name
0,1770249a5a2e770ca90628434d4934b1,7406077cb3ef2783f260ac5ddf06f884,3407,1.0,Route 18
1,1770249a5a2e770ca90628434d4934b1,d9f1f0a4d53997858a48a340772c75a3,3408,0.0,Route 21
2,1770249a5a2e770ca90628434d4934b1,3c36b0976b9cd77a0862224d842fea5f,3408,0.0,Route 21
3,1770249a5a2e770ca90628434d4934b1,a26bcf0a35ac9342d82435bb064f8700,3408,0.0,Route 21
4,1770249a5a2e770ca90628434d4934b1,77b979dca36b0f4fd305210cee23f6a1,3408,0.0,Route 21
...,...,...,...,...,...
98814,c4092405159366c705b62df938293a4e,9f5b9be0653ea22d7c705fb8ccecf20f,Orange County Line,0.0,None
98815,c4092405159366c705b62df938293a4e,c9284435d85c3cf9a5fa5afacbcfb5f0,Antelope Valley Line,1.0,None
98816,c4092405159366c705b62df938293a4e,23bdf6228677587ad66658b818f8f3c8,Orange County Line,1.0,None
98817,c4092405159366c705b62df938293a4e,633996f6d703cbff7838099bc4e8c6e0,San Bernardino Line,0.0,None


# concatanate error?

In [4]:
import datetime
import geopandas as gpd
import gcsfs
import pandas as pd

from dask import delayed, compute
from pathlib import Path
from typing import Literal

fs = gcsfs.GCSFileSystem()

OPERATOR_COLS = ["schedule_gtfs_dataset_key", "name",
                 "organization_source_record_id", "organization_name",
                 "base64_url", "caltrans_district"]
STOP_PAIR_COLS = ["stop_pair", "stop_pair_name"] 
ROUTE_DIR_COLS = ["route_id", "direction_id"]

In [44]:
def concatenate_datasets_across_dates(
    gcs_bucket: str,
    dataset_name: Literal["speeds_route_dir_segments", "speeds_route_dir"],
    date_list: list,
    data_type: Literal["df", "gdf"] = "gdf",
    get_pandas: bool = True,
    **kwargs
) -> pd.DataFrame:
    """
    Concatenate parquets across all months of available data.
    """  
    if data_type == "gdf":
        dfs = [
            delayed(gpd.read_parquet)(
                f"{gcs_bucket}{dataset_name}_{d}.parquet",
                **kwargs
            ).assign(
                service_date = pd.to_datetime(d)
            ) for d in date_list
        ]
        return dfs #  debug
    else:
        dfs = [
            delayed(pd.read_parquet)(
                f"{gcs_bucket}{dataset_name}_{d}.parquet",
                **kwargs
            ).assign(
                service_date = pd.to_datetime(d)
            ) for d in date_list
        ]
        return dfs
    df = delayed(pd.concat)(
        dfs, axis=0, ignore_index=True
    ) 
    
    if get_pandas:
        df = compute(df)[0]    
    
    return df

In [6]:
segment_type = "speedmap_segments"

In [7]:
dict_inputs = GTFS_DATA_DICT[segment_type]

SPEED_FILE = dict_inputs["stage4"]
MAX_SPEED = dict_inputs["max_speed"]

# These are the grouping columns (list) to use for the shape and route-dir aggregation
SHAPE_STOP_COLS = [*dict_inputs["shape_stop_cols"]]
STOP_PAIR_COLS = [*dict_inputs["stop_pair_cols"]]

SHAPE_SEG_FILE = dict_inputs["shape_stop_single_segment"]
ROUTE_SEG_FILE = dict_inputs["route_dir_single_segment"]

In [48]:
(OPERATOR_COLS + SHAPE_STOP_COLS + 
               STOP_PAIR_COLS + ROUTE_DIR_COLS + [
                   "trip_instance_key", "speed_mph", 
                   "meters_elapsed", "sec_elapsed", 
                   "time_of_day"])

['schedule_gtfs_dataset_key',
 'name',
 'organization_source_record_id',
 'organization_name',
 'base64_url',
 'caltrans_district',
 'shape_array_key',
 'shape_id',
 'route_id',
 'route_short_name',
 'stop_pair',
 'stop_pair_name',
 'segment_id',
 'route_id',
 'direction_id',
 'trip_instance_key',
 'speed_mph',
 'meters_elapsed',
 'sec_elapsed',
 'time_of_day']

In [45]:
df = time_series_utils.concatenate_datasets_across_dates(
# df = concatenate_datasets_across_dates(
    SEGMENT_GCS,
    SPEED_FILE,
    [analysis_date],
    data_type  = "df",
    get_pandas = True,
    columns = (OPERATOR_COLS + SHAPE_STOP_COLS + 
               STOP_PAIR_COLS + ROUTE_DIR_COLS + [
                   "trip_instance_key", "speed_mph", 
                   "meters_elapsed", "sec_elapsed", 
                   "time_of_day"]),
    filters = [[("speed_mph", "<=", MAX_SPEED)]]
)

ArrowInvalid: No match for FieldRef.Name(name) in trip_instance_key: string
stop_sequence: int64
stop_sequence1: double
shape_array_key: string
stop_meters: double
arrival_time: timestamp[ns]
arrival_time_sec: int64
subseq_arrival_time_sec: double
subseq_stop_meters: double
meters_elapsed: double
sec_elapsed: double
speed_mph: double
schedule_gtfs_dataset_key: string
route_id: string
direction_id: double
route_short_name: string
scheduled_service_minutes: double
sched_rt_category: dictionary<values=string, indices=int32, ordered=0>
time_of_day: string
shape_id: string
segment_id: string
stop_pair_name: string
stop_pair: string
__index_level_0__: int64
__fragment_index: int32
__batch_index: int32
__last_in_fragment: bool
__filename: string

In [46]:
df[0]

Delayed('assign-ab39c5c3-f043-4f19-af11-cd77d61469bc')

In [39]:
path = f"{SEGMENT_GCS}{SPEED_FILE}_{analysis_date}.parquet"

In [40]:
df = pd.read_parquet(path)

In [41]:
df.columns

Index(['trip_instance_key', 'stop_sequence', 'stop_sequence1',
       'shape_array_key', 'stop_meters', 'arrival_time', 'arrival_time_sec',
       'subseq_arrival_time_sec', 'subseq_stop_meters', 'meters_elapsed',
       'sec_elapsed', 'speed_mph', 'schedule_gtfs_dataset_key', 'route_id',
       'direction_id', 'route_short_name', 'scheduled_service_minutes',
       'sched_rt_category', 'time_of_day', 'shape_id', 'segment_id',
       'stop_pair_name', 'stop_pair'],
      dtype='object')

In [42]:
df

,trip_instance_key,stop_sequence,stop_sequence1,shape_array_key,stop_meters,arrival_time,arrival_time_sec,subseq_arrival_time_sec,subseq_stop_meters,meters_elapsed,...,route_id,direction_id,route_short_name,scheduled_service_minutes,sched_rt_category,time_of_day,shape_id,segment_id,stop_pair_name,stop_pair
0,00008bb8a00ac0aaf3d2fa40311e831d,2,2.0,03fcaf8299c7e0ec22877e93549077ce,138.361,2024-08-14 18:43:16,67396,67397.0,267.398,129.037,...,24,0.0,24,49.0,schedule_and_vp,PM Peak,2401,15147-15162-1,Jackson St & Fillmore St__Jackson St & Steiner St,15147__15162
1,00008bb8a00ac0aaf3d2fa40311e831d,3,3.0,03fcaf8299c7e0ec22877e93549077ce,267.398,2024-08-14 18:43:17,67397,67400.0,546.608,279.210,...,24,0.0,24,49.0,schedule_and_vp,PM Peak,2401,15162-15160-1,Jackson St & Steiner St__Jackson St & Scott St,15162__15160
2,00008bb8a00ac0aaf3d2fa40311e831d,4,4.0,03fcaf8299c7e0ec22877e93549077ce,546.608,2024-08-14 18:43:20,67400,67402.0,693.431,146.823,...,24,0.0,24,49.0,schedule_and_vp,PM Peak,2401,15160-15145-1,Jackson St & Scott St__Jackson St & Divisadero St,15160__15145
3,00008bb8a00ac0aaf3d2fa40311e831d,5,5.0,03fcaf8299c7e0ec22877e93549077ce,693.431,2024-08-14 18:43:22,67402,67405.0,905.913,212.482,...,24,0.0,24,49.0,schedule_and_vp,PM Peak,2401,15145-14413-1,Jackson St & Divisadero St__Divisadero St & Cl...,15145__14413
4,00008bb8a00ac0aaf3d2fa40311e831d,6,6.0,03fcaf8299c7e0ec22877e93549077ce,905.913,2024-08-14 18:43:25,67405,67407.0,1129.279,223.366,...,24,0.0,24,49.0,schedule_and_vp,PM Peak,2401,14413-14410-1,Divisadero St & Clay St__Divisadero St & Calif...,14413__14410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3026347,ff2f799a2505ee42ae33cb94a6a3a7a1,20,20.0,15764e14eab1791e9506a62095e9f001,7111.868,2024-08-14 06:36:30,23790,23486.0,7322.164,210.296,...,40,0.0,40,23.0,schedule_and_vp,Early AM,114235,62034-65913-1,Shoreline & Middlefield__Shoreline & Sutter Creek,62034__65913
3026348,ff2f799a2505ee42ae33cb94a6a3a7a1,21,21.0,15764e14eab1791e9506a62095e9f001,7322.164,2024-08-14 06:31:26,23486,22941.0,7698.816,376.652,...,40,0.0,40,23.0,schedule_and_vp,Early AM,114235,65913-61417-1,Shoreline & Sutter Creek__Shoreline & Wright,65913__61417
3026349,ff2f799a2505ee42ae33cb94a6a3a7a1,22,22.0,15764e14eab1791e9506a62095e9f001,7698.816,2024-08-14 06:22:21,22941,22941.0,8570.730,871.914,...,40,0.0,40,23.0,schedule_and_vp,Early AM,114235,61417-65912-1,Shoreline & Wright__Villa & Franklin,61417__65912
3026350,ff2f799a2505ee42ae33cb94a6a3a7a1,23,23.0,15764e14eab1791e9506a62095e9f001,8570.730,2024-08-14 06:22:21,22941,22941.0,8793.525,222.795,...,40,0.0,40,23.0,schedule_and_vp,Early AM,114235,65912-61426-1,Villa & Franklin__Villa & Castro,65912__61426
